## Large Scrape of Flow Data Into GCS

In [1]:
import json
import sys
import os
from datetime import datetime
sys.path.append('/Users/gisaac/Documents/GitHub/river_flow_notebooks/')
from scraping_functions import HydroScraper
import pandas as pd

In [6]:
def run_scrape(start_time, end_time, idx=0, base_path="meta_temp/sentinel_11_03"):
    dir_list = os.listdir(base_path)
    for i in range(idx, len(dir_list)):
        print("Now getting data for " + str(i))
        print(dir_list[i])
        try:
            h = HydroScraper(start_time, end_time, os.path.join(base_path, dir_list[i]), asos_bq_table="asos_weather")
            h.combine_data()
            if "snotel" in h.meta_data:
                h.combine_snotel_with_df()
            else:
                h.final_df = h.joined_df
            sentinel_df = pd.read_csv("/Users/gisaac/Downloads/sent_data.csv")
            h.combine_sentinel(sentinel_df, h.meta_data["sentinel_2_tile_id"])
            file_upload_id = h.meta_data["site_number"]
            h.bq_connect.upload_file_to_gcs(h.final_df, file_upload_id)
            h.final_df["usgs_id"] = h.meta_data["site_number"]
            print(h.final_df.columns)
            h.write_final_df_to_bq("final_flow_data")
        except KeyError as e:
            print(e)
            print("Failed " + dir_list[i])

    pass

In [ ]:
run_scrape(datetime(2024, 1, 1), datetime(2024, 1, 2), idx=6)

Now getting data for 6
073813375.json
Getting request from USGS
http://waterservices.usgs.gov/nwis/iv/?format=rdb,1.0&sites=73813375&startDT=2024-01-01&endDT=2024-01-02&parameterCd=00060,00065,00045&siteStatus=all
Request finished
'tz_cd'
Failed 073813375.json
Now getting data for 7
06899900.json
Getting request from USGS
http://waterservices.usgs.gov/nwis/iv/?format=rdb,1.0&sites=06899900&startDT=2024-01-01&endDT=2024-01-02&parameterCd=00060,00065,00045&siteStatus=all
Request finished
Getting request from ASOS
https://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?station=KTVK&data=tmpf&data=dwpf&data=relh&data=feel&data=sknt&data=sped&data=alti&data=mslp&data=drct&data=ice_accretion_1hr&data=p01m&data=vsby&data=gust&data=skyc1&data=peak_wind_gust&data=snowdepth&year1=2024&month1=1&day1=1&year2=2024&month2=1&day2=4&tz=Etc%2FUTC&format=onlycomma&latlon=no&elev=no&missing=M&trace=T&direct=no&report_type=3&report_type=4


/Users/gisaac/Documents/GitHub/river_flow_notebooks/scraping_functions.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["datetime"] = df["datetime"].map(lambda x: old_timezone.localize(datetime.strptime(x, "%Y-%m-%d %H:%M")).astimezone(new_timezone))
/Users/gisaac/Documents/GitHub/river_flow_notebooks/scraping_functions.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["cfs"] = pd.to_numeric(df['cfs'], errors='coerce')
/Users/gisaac/Documents/GitHub/river_flow_notebooks/scraping_functions.py:73

    station             valid  tmpf  dwpf   relh   feel  sknt   sped   alti  \
0       TVK  2024-01-01 00:15  30.2  24.8  80.07  21.28   9.0  10.35  30.23   
1       TVK  2024-01-01 00:35  30.2  24.8  80.07  20.71  10.0  11.50  30.24   
2       TVK  2024-01-01 00:55  30.2  24.8  80.07  20.71  10.0  11.50  30.24   
3       TVK  2024-01-01 01:15  30.2  23.0  74.26  19.71  12.0  13.80  30.25   
4       TVK  2024-01-01 01:35  30.2  23.0  74.26  21.28   9.0  10.35  30.25   
..      ...               ...   ...   ...    ...    ...   ...    ...    ...   
211     TVK  2024-01-03 22:35  37.4  24.8  60.04  32.03   6.0   6.90  30.15   
212     TVK  2024-01-03 22:55  35.6  24.8  64.46  29.87   6.0   6.90  30.15   
213     TVK  2024-01-03 23:15  33.8  24.8  69.25  28.53   5.0   5.75  30.16   
214     TVK  2024-01-03 23:35  33.8  24.8  69.25  26.33   8.0   9.20  30.17   
215     TVK  2024-01-03 23:55  32.0  24.8  74.44  26.40   5.0   5.75  30.18   

    mslp    drct ice_accretion_1hr  p01m  vsby gust

/Users/gisaac/anaconda3/lib/python3.11/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/Users/gisaac/anaconda3/lib/python3.11/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Index(['hour_updated', 'p01m', 'valid', 'tmpf', 'dwpf', 'ice_accretion_1hr',
       'mslp', 'drct', 'sped', 'alti', 'relh', 'sknt', 'feel', 'vsby', 'gust',
       'skyc1', 'peak_wind_gust', 'snowdepth', 'station_id', 'index',
       'datetime', 'precip_usgs', 'cfs', 'height', 'sensing_time', 'base_url',
       'usgs_id'],
      dtype='object')
LoadJob<project=hydro-earthnet-db, location=EU, id=dff44a0f-e972-4e77-857a-243cabc664b1>
Now getting data for 8
07263880.json
Getting request from USGS
http://waterservices.usgs.gov/nwis/iv/?format=rdb,1.0&sites=07263880&startDT=2024-01-01&endDT=2024-01-02&parameterCd=00060,00065,00045&siteStatus=all
Request finished
'cfs'
Failed 07263880.json
Now getting data for 9
09296800.json
Getting request from USGS
http://waterservices.usgs.gov/nwis/iv/?format=rdb,1.0&sites=09296800&startDT=2024-01-01&endDT=2024-01-02&parameterCd=00060,00065,00045&siteStatus=all


...

LoadJob<project=hydro-earthnet-db, location=EU, id=31a61ff5-0152-439d-8a4f-e66c11fbf23e>
ASOS data written to BigQuery
Index(['hour_updated', 'p01m', 'valid', 'tmpf', 'dwpf', 'ice_accretion_1hr',
       'mslp', 'drct', 'sped', 'alti', 'relh', 'sknt', 'feel', 'vsby', 'gust',
       'skyc1', 'peak_wind_gust', 'snowdepth', 'station_id', 'index',
       'datetime', 'cfs', 'height', 'sensing_time', 'base_url', 'usgs_id'],
      dtype='object')
LoadJob<project=hydro-earthnet-db, location=EU, id=dc82a2f2-b3d3-4585-9177-32d32e0d6482>
Now getting data for 23
03111500.json
Getting request from USGS
http://waterservices.usgs.gov/nwis/iv/?format=rdb,1.0&sites=03111500&startDT=2024-01-01&endDT=2024-01-02&parameterCd=00060,00065,00045&siteStatus=all
Request finished
Getting request from ASOS
https://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?station=HLG&data=tmpf&data=dwpf&data=relh&data=feel&data=sknt&data=sped&data=alti&data=mslp&data=drct&data=ice_accretion_1hr&data=p01m&data=vsby&data=gus

/Users/gisaac/Documents/GitHub/river_flow_notebooks/scraping_functions.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["datetime"] = df["datetime"].map(lambda x: old_timezone.localize(datetime.strptime(x, "%Y-%m-%d %H:%M")).astimezone(new_timezone))
/Users/gisaac/Documents/GitHub/river_flow_notebooks/scraping_functions.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["cfs"] = pd.to_numeric(df['cfs'], errors='coerce')
/Users/gisaac/Documents/GitHub/river_flow_notebooks/scraping_functions.py:73

    station             valid  tmpf  dwpf   relh   feel   sknt   sped   alti  \
0       HLG  2024-01-01 00:53  37.0  34.0  88.78  32.33   5.00   5.75  29.92   
1       HLG  2024-01-01 01:30  37.0  34.0  88.78  32.33   5.00   5.75  29.92   
2       HLG  2024-01-01 01:46  38.0  34.0  85.35  32.76   6.00   6.90  29.92   
3       HLG  2024-01-01 01:53  38.0  34.0  85.35  32.76   6.00   6.90  29.92   
4       HLG  2024-01-01 02:21  38.0  34.0  85.35  32.76   6.00   6.90  29.92   
..      ...               ...   ...   ...    ...    ...    ...    ...    ...   
110     HLG  2024-01-03 19:53  41.0  26.0  54.84  33.84  11.00  12.65  29.96   
111     HLG  2024-01-03 20:53  38.0  25.0  59.13  32.09   7.00   8.05  29.96   
112     HLG  2024-01-03 21:53  38.0  25.0  59.13  32.76   6.00   6.90  29.96   
113     HLG  2024-01-03 22:53  36.0  25.0  63.98  33.26   3.00   3.45  29.96   
114     HLG  2024-01-03 23:53  35.0  25.0  66.57  29.96   5.00   5.75  29.97   

        mslp    drct ice_accretion_1hr 

/Users/gisaac/anaconda3/lib/python3.11/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/Users/gisaac/anaconda3/lib/python3.11/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


LoadJob<project=hydro-earthnet-db, location=EU, id=c5ab1345-f7bb-4abe-9a91-9fc6b7fe9ede>
ASOS data written to BigQuery
Index(['hour_updated', 'p01m', 'valid', 'tmpf', 'dwpf', 'ice_accretion_1hr',
       'mslp', 'drct', 'sped', 'alti', 'relh', 'sknt', 'feel', 'vsby', 'gust',
       'skyc1', 'peak_wind_gust', 'snowdepth', 'station_id', 'index',
       'datetime', 'cfs', 'height', 'sensing_time', 'base_url', 'usgs_id'],
      dtype='object')
